#GDrive setup

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/MyDrive/Uni/AAIB/AAIB

#Libraries

In [ ]:
!pip install matplotlib==3.1.3

In [ ]:
!pip install focal-loss

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from google.colab import drive
import tensorflow as tf
import numpy as np

import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array,array_to_img
from sklearn.metrics import confusion_matrix
from keras.callbacks import *
import random
from keras import backend as K
from PIL import Image
import cv2
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
import albumentations as A
from itertools import combinations
import json
from sklearn.utils import class_weight
from focal_loss import SparseCategoricalFocalLoss
from sklearn.metrics import classification_report
from typing import Tuple
import scipy
from skimage.measure import label as label_fn
from skimage import filters
import albumentations as A
from focal_loss import SparseCategoricalFocalLoss
from sklearn.model_selection import StratifiedKFold

#Env Setup

In [ ]:
SEED = 4224
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

labels_path = 'data/labels_train_clean.csv'
all_data_no_duplicates_path = 'data/train_all_no_duplicates'
clean_data_path = 'data/train_clean/'
noisy_data_path = 'data/train_noisy/'

train_percentage = 0.8
validation_percentage = 0.15
test_percentage = 0.2
img_size = (224,224)
batch_size = 16

# Cyclical LR

In [ ]:
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

#Data Generator

In [ ]:
class CustomGenerator(tf.keras.utils.Sequence):
  """
    CustomGenerator inheriting from tf.keras.utils.Sequence.

    We have to implement 3 main methods:
      - __init__: save dataset params like directory, filenames, etc.
      - __len__: return the total number of samples in the dataset (number of batches)
      - __getitem__: return a single batch of paired images masks
  """

  def __init__(self, 
               dataframe, # dataframe of the dataset  
               base_path,
               preprocessing_function=None, # Preprocessing function (e.g., the one used for transfer learning)
               batch_size=16, # Batch size
               out_shape = (100,100),
               shuffle=False,
               categorical = True,
               augment = False,
               seed = SEED,
               flow_from_directory = True,
               preprocess_input = False):
    
    # Get all filenames
    if isinstance(base_path, Tuple):
      self.filenames = []
      for p in base_path:

        paths = self.folderToPaths(p, full_path = False)

        for pa in paths:
          if pa in set(dataframe.file):
            self.filenames.append(os.path.join(p, pa))


    else:
        self.filenames = [os.path.join(base_path, img_path) for img_path in list(dataframe.file)]

    self.labels = tfk.utils.to_categorical(list(dataframe.label)) if categorical else list(dataframe.label)

    # Set indices list in [0, len(subset_filenames)]
    self.indices = np.arange(len(self.filenames))

    # Save dataset parameters as class attributes
    self.base_path = base_path
    self.preprocessing_function = preprocessing_function
    self.out_shape = out_shape
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.augment = augment
    self.seed = seed
    self.flow_from_directory =flow_from_directory
    self.data_augmentation = A.Compose([
    A.RandomBrightnessContrast(brightness_limit = 0.05, contrast_limit=0.05, p=0.5),
    A.ShiftScaleRotate(p = 0.8, rotate_limit = 20, scale_limit = 0.3, border_mode =  cv2.BORDER_CONSTANT, value = 0),
    A.CLAHE(p=0.2)
    ])
    self.preprocess_input = preprocess_input

    if not self.flow_from_directory:
      self.images = self.load_all_imgs()

  def augmentation(self, images):
    return self.data_augmentation(image = images)


  def __filterNoisyOnClahe(self, image):
    clahe = cv2.createCLAHE(clipLimit = 300, tileGridSize = (50, 50))
    im1 = cv2.resize(image, (400, 400))
    im1 = scipy.ndimage.gaussian_laplace(im1, sigma = 6)
    im1 = clahe.apply(im1)
    var1 = np.var(im1)
    if var1 > 800:
      image= cv2.medianBlur(image, ksize=5)
      return scipy.ndimage.uniform_filter(image, size=3)
    else:
      return image


  def __sharpenImage(self, image):
    sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])

    sharpened = cv2.filter2D(image, -1, sharpen_kernel)

    return sharpened

  def __invert_image(self, img):

    otsu_thresh = filters.threshold_otsu(img)
    masked_image = (img > otsu_thresh) * 1.0
    valsROI1, _ = np.histogram(masked_image[220:244, 220:244], bins=2, range=(0, 1))
    valsROI2, _ = np.histogram(masked_image[0:25, 0:25], bins=2, range=(0, 1))
    valsROI3, _ = np.histogram(masked_image[0:25, 220:244], bins=2, range=(0, 1))
    valsROI4, _ = np.histogram(masked_image[220:244, 0:25], bins=2, range=(0, 1))
    
    valsTot = valsROI1 + valsROI2 + valsROI3 + valsROI4
    
    labels = label_fn(masked_image)

    if len(np.unique(labels)) < 100:
        if valsTot[0] > valsTot[1]:
            return img
        else:
            return 255 - img
    return img

  def __filterBlurred(self, image):
    clahe = cv2.createCLAHE(clipLimit = 1.8, tileGridSize = (4, 4))
    minThresh = 2
    im1 = cv2.resize(image, (400, 400))
    im1 = scipy.ndimage.gaussian_laplace(im1, sigma = 2)
    im1 = clahe.apply(im1)
    _, im1 = cv2.threshold(im1, minThresh, 255, cv2.THRESH_BINARY)
    var1 = np.var(im1)
    if var1 < 5:
        return self.__sharpenImage(image)
    else:
        return self.__sharpenImage(image)

  def __filterUnderexposed(self, image):
    im1 = cv2.resize(image, (400, 400))
    mean1 = np.mean(im1)
    if mean1 < 71:
      clahe = cv2.createCLAHE(clipLimit = 2, tileGridSize = (2, 2))
      image = clahe.apply(image)
      return image
    else:
      return image


  def preprocess(self, image):

    image = self.__invert_image(image)
    image = self.__filterUnderexposed(image)
    image = self.__filterNoisyOnClahe(image)
    image = self.__filterBlurred(image)
   

    return image

  def __len__(self):
    # Return the length of the dataset (number of batches)
    # that is given by #images // batch_size
    return len(self.filenames) // self.batch_size

  def on_epoch_start(self):
    # Shuffle indices after each epoch
    if self.shuffle == True:
        np.random.shuffle(self.indices)

  def load_all_imgs(self):
      images = []
      for f in self.filenames:
        image = cv2.imread(f, 0)
        image = cv2.resize(image, (self.out_shape))
        if self.preprocess_input:
          image = self.preprocess(image)
        images.append(image)

      return np.array(images)

  def get_image_and_label(self, index):

    if not self.flow_from_directory:
      image = self.images[index]
      if self.augment:
        image = self.augmentation(image)
      image = np.squeeze(image)
      curr_label = self.labels[index]
    else:
      curr_filename = self.filenames[index] # Get filename at index
      curr_label = self.labels[index]
      image = cv2.imread(curr_filename, 0)
      image = cv2.resize(image, (self.out_shape))
      if self.preprocess_input:
        image = self.preprocess(image)

      if self.augment:
        image = self.augmentation(image)['image']



    return image, curr_label

  def __getitem__(self, index):
    # In this function we generate a batch (of size self.batch_size) of images and corresponding masks
    
    # Get 'self.batch_size' indices
    current_indices = self.indices[index*self.batch_size:(index*self.batch_size)+self.batch_size]

    """if len(current_indices) == 0:
      current_indices = self.indices[len(self.indices)-self.batch_size:len(self.indices)]"""

    # Init lists that will contain images and masks
    batch_images = []
    batch_labels = []

    # Cycle over the indices
    for idx in current_indices:
      # Get single image/mask at index 'idx'
      image, label = self.get_image_and_label(idx)

      # Apply the preprocessing function
      if self.preprocessing_function is not None:
        image = self.preprocessing_function(image)

      # Append both image and mask (with added batch dimension) to the corresponding batch lists
      batch_images.append(np.expand_dims(image, 0))
      batch_labels.append(label)
     
    # Finally, obtain a final batch by concatenating all the images over the batch dimension
    batch_images = np.concatenate(batch_images, axis=0)
    batch_labels = np.array(batch_labels)

    return batch_images, batch_labels


  def folderToPaths(
        self,
        full_img_dir,
        full_path = True
):

    x_paths_list = []

    full_img_dir = full_img_dir

    for full in os.listdir(full_img_dir):
         if full_path:
            x_paths_list.append(os.path.join(full_img_dir, full))
         else:
          x_paths_list.append(full)
    
    x_paths_list.sort()
    return x_paths_list

#Data Loading

In [ ]:
def encode(x):
  if x == 'N':
    return 0
  elif x == 'P':
    return 1
  else:
    return 2

In [ ]:
def folderToPaths(
        full_img_dir,
        full_path = True
):

    x_paths_list = []

    full_img_dir = full_img_dir

    for full in os.listdir(full_img_dir):
         if full_path:
            x_paths_list.append(os.path.join(full_img_dir, full))
         else:
          x_paths_list.append(full)
    
    x_paths_list.sort()
    return x_paths_list

In [ ]:
labelsDF = pd.read_csv(labels_path)
display(labelsDF.head(20))

In [ ]:
labelsDF.label = labelsDF.label.apply(lambda x: encode(x))
display(labelsDF.head(20))

In [ ]:
all_data_no_duplicates_path_list = folderToPaths(full_img_dir = all_data_no_duplicates_path)

In [ ]:
train_val, test = train_test_split(labelsDF, test_size = test_percentage, shuffle = True, stratify = labelsDF.label)
train, val = train_test_split(train_val, test_size = validation_percentage, shuffle = True, stratify = train_val.label)

In [ ]:
train_gen = CustomGenerator(dataframe = train, base_path = 'data/train_all_no_duplicates', batch_size = batch_size, out_shape = img_size, shuffle = True, categorical=False, flow_from_directory=False, preprocess_input = True)
valid_gen = CustomGenerator(dataframe = val, base_path = 'data/train_all_no_duplicates', batch_size = batch_size, out_shape = img_size, shuffle = True, categorical=False, flow_from_directory=False, preprocess_input = True)
test_gen = CustomGenerator(dataframe = test, base_path = 'data/train_all_no_duplicates', batch_size = batch_size, out_shape = img_size, shuffle = True, categorical=False, flow_from_directory=False, preprocess_input = True)

In [ ]:
dataset_labels = np.array(list(set(labelsDF.label)), dtype=int)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = dataset_labels,
                                                 y = train.label)
class_weights

# Residual Network developed from scratch

##Model Definition

In [ ]:
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, concatenate, BatchNormalization, Reshape
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization,Concatenate, Resizing
from keras import regularizers

In [ ]:
def ResBs_Conv(block_input, num_filters): 
   
    # 0. Filter Block input and BatchNormalization
    block_input = Conv2D(num_filters, kernel_size=7, strides=2,  padding='same')(block_input) 
    block_input = BatchNormalization()(block_input)

    # 1. First Convolutional Layer
    conv1 = Conv2D(filters=num_filters, kernel_size=7, padding='same')(block_input)
    norm1 = BatchNormalization()(conv1)
    relu1 = Activation('relu')(norm1)  
    dropout = Dropout(0.2)(relu1)
    
    # 2. Second Convolutional Layer 
    conv2 = Conv2D(num_filters, kernel_size=7, padding='same')(dropout)
    norm2 = BatchNormalization()(conv2)

    # 3. Summing Layer (adding a residual connection)
    sum = Add()([block_input, norm2])
    
    # 4. Activation Layer
    relu2 = Activation('relu')(sum)
    
    return relu2 

In [ ]:
def ResBs_Identity(block_input, num_filters): 

    # 1. First Convolutional Layer
    conv1 = Conv2D(filters=num_filters, kernel_size=7, padding= 'same')(block_input)
    norm1 = BatchNormalization()(conv1)
    relu1 = Activation('relu')(norm1)    
    dropout = Dropout(0.2)(relu1)
    
    # 2. Second Convolutional Layer 
    conv2 = Conv2D(num_filters, kernel_size=7, padding= 'same')(dropout)
    norm2 = BatchNormalization()(conv2)

    # 3. Summing Layer (adding a residual connection)
    sum = Add()([block_input, norm2])
    
    # 4. Activation Layer
    relu2 = Activation('relu')(sum)
    
    return relu2 

In [ ]:
def build_model(input_shape, N):
  inputs = tfk.Input(shape=input_shape)
  
  conv1 = tfkl.Conv2D(
          filters=32,
          kernel_size=3,
          padding = 'same',
          activation = 'relu',
          kernel_initializer = tfk.initializers.HeUniform(SEED)
      )(inputs)
  bn1 = tfkl.BatchNormalization()(conv1) 
  relu1 = Activation('relu')(bn1) 
  pool1 = MaxPooling2D()(relu1)


  #left side
  conv2_1 = tfkl.Conv2D(
          filters=64,
          kernel_size=3,
          padding = 'same',
          activation = 'relu',
          kernel_initializer = tfk.initializers.HeUniform(SEED)
      )(pool1)
  bn2_1 = BatchNormalization()(conv2_1)
  relu2 = Activation('relu')(bn2_1) 

  conv2_1 = tfkl.Conv2D(
          filters=64,
          kernel_size=3,
          padding = 'same',
          activation = 'relu',
          kernel_initializer = tfk.initializers.HeUniform(SEED)
      )(relu2)
  bn2_1 = BatchNormalization()(conv2_1)
  relu2 = Activation('relu')(bn2_1) 

  conv2_1 = tfkl.Conv2D(
          filters=64,
          kernel_size=3,
          padding = 'same',
          activation = 'relu',
          kernel_initializer = tfk.initializers.HeUniform(SEED)
      )(relu2)
  bn2_1 = BatchNormalization()(conv2_1)  

  bn2_1 = tfkl.Dropout(0.2, seed=SEED)(bn2_1)

  #right side
  conv2_2 = tfkl.Conv2D(
          filters=64,
          kernel_size=3,
          padding = 'same',
          activation = 'relu',
          kernel_initializer = tfk.initializers.HeUniform(SEED)
      )(pool1)
  bn2_2 = BatchNormalization()(conv2_2)

  bn2_2 = tfkl.Dropout(0.3, seed=SEED)(bn2_2)

  #addition
  sum = Add()([bn2_1, bn2_2])
  relu = Activation('relu')(sum)

  ResNet = tfkl.Dropout(0.2, seed=SEED)(relu)

  filters=64
  '''we also tried
  filters=32'''
  
  M = int((N - 2)/2)
  for i in range(M): 
    filters = filters * 2
    # define N-th ResBs block
    ResNet = ResBs_Conv(ResNet, filters)
    ResNet = ResBs_Identity(ResNet, filters)
    
  y = tf.keras.layers.GlobalAveragePooling2D()(ResNet)

  y = tf.keras.layers.Dense(512, activation='relu', kernel_initializer = tfk.initializers.HeUniform(SEED))(y)
  y = tfkl.Dropout(0.1, seed=SEED)(y)

  outputs = tf.keras.layers.Dense(3, activation='softmax', kernel_initializer = tfk.initializers.GlorotUniform(SEED))(y)

  # Connect input and output through the Model class
  model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

  # Compile the model

  model.compile(loss=SparseCategoricalFocalLoss(class_weight = class_weights, gamma=2), optimizer=tfk.optimizers.Adam(learning_rate = 1e-4), metrics='accuracy')

  return model

In [ ]:
#trials
model = build_model((224,224,1), 6)
model = build_model((224,224,1), 4)

In [ ]:
res_model = build_model((224,224,1), 8)
res_model.summary()

In [ ]:
tfk.utils.plot_model(res_model)

In [ ]:
del res_model

##Training

In [ ]:
training_samples = int(len(train_gen)*batch_size)
step_size = 6*training_samples // batch_size

clr = CyclicLR(
    mode='triangular',
    base_lr=1e-5, 
    max_lr=1e-4,
    step_size= step_size)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = res_model.fit(train_gen,
    epochs = 20,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor= 'val_accuracy', mode='max', patience=10, restore_best_weights=True), clr, model_checkpoint_callback],
).history

In [ ]:
# The model weights (that are considered the best) are loaded into the
# model.
res_model.load_weights(checkpoint_filepath)

In [ ]:
# Plot the training
plt.figure(figsize=(20,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Category Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(20,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

##Test

In [ ]:
# Predict the test set with the CNN
predictions = res_model.predict(test_gen)

In [ ]:
y = np.argmax(tfk.utils.to_categorical(list(test.label))[:-2], axis = -1)
pred = np.argmax(predictions, axis=-1)
target_names = ['N', 'P',' T']
cm = confusion_matrix(y, pred, normalize="true")



# Compute the classification metrics
accuracy = accuracy_score(y, pred)
precision = precision_score(y, pred, average='macro')
recall = recall_score(y, pred, average='macro')
f1 = f1_score(y, pred, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))
print(classification_report(y, pred, target_names=target_names, digits=4))
# Plot the confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm.T, xticklabels=[0,1,2], yticklabels=[0,1,2])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
import seaborn as sns

# Normalise
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=list(target_names), yticklabels=list(target_names))
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)

In [ ]:
pb_model_dir = "Model/denseNET_ft"
h5_model = "/gdrive/MyDrive/bestResNet.h5"

tf.keras.models.save_model(res_model, h5_model)

##KFold crossvalidation

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = SEED)

In [ ]:
print(skf.split(train_val.file.to_numpy(), train_val.label.to_numpy()))

Since the training required a lot of time, in order to perform cross-validation we saved the indices of the stratified k-fold and then trained a model at a time to avoid exceeding colab GPU time.

In [ ]:
for i, (train_index, valid_index) in enumerate(skf.split(train_val.file.to_numpy(), train_val.label.to_numpy())):
  print (i,train_index, valid_index)
  model_directory = '/gdrive/MyDrive/Uni/AAIB'
  trainname = 'train_idx' + str(i)
  trainname_chosen = os.path.join(model_directory, trainname)
  np.save(trainname_chosen, train_index)
  testname = 'test_idx' + str(i)
  testname_chosen = os.path.join(model_directory, testname)
  np.save(testname_chosen, valid_index)

In [ ]:
train_index = np.load('/gdrive/MyDrive/Uni/AAIB/train_idx0.npy')
valid_index = np.load('/gdrive/MyDrive/Uni/AAIB/test_idx0.npy')

In [ ]:
trainDF = train_val.filter(items= train_index, axis=0)
validDF = train_val.filter(items= valid_index, axis=0)

train_gen = CustomGenerator(dataframe = trainDF, base_path = 'data/train_all_no_duplicates', batch_size = batch_size, out_shape = img_size, shuffle = True, flow_from_directory=True, preprocess_input = False, categorical = False, augment = False)
valid_gen = CustomGenerator(dataframe = validDF, base_path = 'data/train_all_no_duplicates', batch_size = batch_size, out_shape = img_size, shuffle = True, flow_from_directory=True, preprocess_input = False, categorical = False, augment = False)

In [ ]:
METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
  ]

In [ ]:
training_samples = int(len(train_gen)*batch_size)
step_size = 6*training_samples // batch_size

clr = CyclicLR(
      mode='triangular',
      base_lr=1e-5, 
      max_lr=1e-4,
      step_size= step_size
      )
model = build_model((224,224,3), 8)
 
epochs = 20

history = model.fit(train_gen,
                    epochs=epochs,  
                    validation_data = valid_gen,
                    callbacks = [tfk.callbacks.EarlyStopping(monitor= 'val_accuracy', mode='max', patience=15, restore_best_weights=True), clr], workers =8, use_multiprocessing = True
                    )

In [ ]:
# Plot the training
plt.figure(figsize=(20,5))
plt.plot(history.history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history.history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Category Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(20,5))
plt.plot(history.history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history.history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

#prediction on test set
predictions = model.predict(test_gen)

y = np.argmax(tfk.utils.to_categorical(list(test.label))[:-2], axis = -1)
pred = np.argmax(predictions, axis=-1)
target_names = ['N', 'P', 'T']
cm = confusion_matrix(y, pred, normalize="true")

# Compute the classification metrics
accuracy = accuracy_score(y, pred)
precision = precision_score(y, pred, average='macro')
recall = recall_score(y, pred, average='macro')
f1 = f1_score(y, pred, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))
print(classification_report(y, pred, target_names=target_names, digits=4))
# Plot the confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm.T, xticklabels=[0,1,2], yticklabels=[0,1,2])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

# Model saving
model_directory = '/gdrive/MyDrive/Uni/AAIB/AAIB/Davide/Kfold'
filename = 'resnet_kFold_0'
filename_chosen = os.path.join(model_directory, filename)
model.save(filename_chosen + '.h5', overwrite = False)
del model

In [ ]:
from keras.models import load_model

# load models 
total_model = 5
model_directory = '/gdrive/MyDrive/Uni/AAIB/AAIB/Davide/Resnet_models/k_fold'
trained_models = list()
for model_n in range(total_model):
  filename = 'resnet_kFold_' + str(model_n)
  filename_chosen = os.path.join(model_directory, filename)
  trained_models.append(load_model(filename_chosen + '.h5'))

In [ ]:
validation_stats = {}
plt.get_current_fig_manager().full_screen_toggle() # toggle fullscreen mode

for idx, model in enumerate(trained_models):  
  predictions = model.predict(test_gen)
  y = np.argmax(tfk.utils.to_categorical(list(test.label))[:-2], axis = -1)
  pred = np.argmax(predictions, axis=-1)
  target_names = ['N', 'P', 'T']
  cm = confusion_matrix(y, pred, normalize="true")


  # Compute the classification metrics
  accuracy = accuracy_score(y, pred)
  precision = precision_score(y, pred, average='macro')
  recall = recall_score(y, pred, average='macro')
  f1 = f1_score(y, pred, average='macro')


  validation_stats['fold_' + str(idx+1) ] = {
      'accuracy' : accuracy,
      'precision' :precision,
      'recall' :recall,
      'f1' :f1,
      'classification_report' :classification_report(y, pred, target_names=target_names, digits=4, output_dict = True)}


  print('Accuracy:',accuracy.round(4))
  print('Precision:',precision.round(4))
  print('Recall:',recall.round(4))
  print('F1:',f1.round(4))
  print(classification_report(y, pred, target_names=target_names, digits=4))
  # Plot the confusion matrix
  plt.figure(figsize=(10,10))
  hm = sns.heatmap(cm.T, xticklabels=[0,1,2], yticklabels=[0,1,2])
  plt.xlabel('True labels')
  plt.ylabel('Predicted labels')
  plt.show()


with open('/gdrive/MyDrive/Uni/AAIB/AAIB/Davide/Kfold/test_stats.json' , 'w') as fp:
    json.dump(validation_stats, fp)

In [ ]:
preds = []

for idx, svm_model in enumerate(trained_models):  
  predictions = svm_model.predict(test_gen)
  y = np.argmax(tfk.utils.to_categorical(list(test.label))[:-2], axis = -1)
  pred = np.argmax(predictions, axis=-1)

  preds.append(pred)

In [ ]:
preds_mv = scipy.stats.mode(preds, axis=0)[0][0]

In [ ]:
y = np.argmax(tfk.utils.to_categorical(list(test.label))[:-2], axis = -1)
pred = preds_mv
target_names = ['N', 'P', 'T']
cm = confusion_matrix(y, pred, normalize="true")


# Compute the classification metrics
accuracy = accuracy_score(y, pred)
precision = precision_score(y, pred, average='macro')
recall = recall_score(y, pred, average='macro')
f1 = f1_score(y, pred, average='macro')

test_stats = {
      'accuracy' : accuracy,
      'precision' :precision,
      'recall' :recall,
      'f1' :f1,
      'classification_report' :classification_report(y, pred, target_names=target_names, digits=4, output_dict = True)}


print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))
print(classification_report(y, pred, target_names=target_names, digits=4))
# Plot the confusion matrix
plt.figure(figsize=(10,10))
hm = sns.heatmap(cm.T, xticklabels=[0,1,2], yticklabels=[0,1,2])
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

hm.get_figure().savefig("ensemble_cm.pdf")

with open('test_stats_ensemble.json' , 'w') as fp:
    json.dump(test_stats, fp)